In [ ]:
'''
SSAFS：applying SSAFS to filter the optimal feature subset of plant disease images
input:features of plant disease images(file is of type csv)
output:best fitness, number of feature subsets, feature subsets
'''
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import datetime
import math

for i in range(10):
    data="grape_feature.csv"
    Data=pd.read_csv(data,na_values='?',header=None)
    Data_x=Data.replace(np.nan, 0)
    Data_x=Data.iloc[:,:-1]
    Data_y=Data.iloc[:,-1]
    #print(Data.shape)
    feat_list=list(Data_x)
    fit=[]
    acc=0
    cnt=0



'''
Salp:population initialisation, 
    fitness ranking, 
    leader and follower position updating,
    assessing the quality of feature subsets
'''
    class Salp():
        def __init__(self,f_list=[]):
            self.f_list=f_list
            self.fitness=None

        def Fitness(self):
            Data=pd.read_csv(data,na_values='?')
            Data=Data.replace(np.nan, 0)
            Data_x=Data.iloc[:,:-1]
            Data_y=Data.iloc[:,-1]
            feat_list=list(Data_x)
            for i,f in enumerate(self.f_list):
                if f==0:Data_x=Data_x.drop([feat_list[i]],axis=1)
            knn=KNeighborsClassifier(n_neighbors=5)
            cv_scores=cross_val_score(knn,Data_x,Data_y,cv=5)
            error_rate=np.mean([1-acc for acc in cv_scores])
            self.f_list=[int(item) for item in self.f_list]
            F=self.f_list.count(1)
            self.fitness=(0.9*error_rate)+(0.1*F/len(self.f_list))

        def Acc(self):
            Data=pd.read_csv(data,na_values='?')
            Data=Data.replace(np.nan, 0)
            Data_x=Data.iloc[:,:-1]
            Data_y=Data.iloc[:,-1]
            feat_list=list(Data_x)
            for i,f in enumerate(self.f_list):
                if f==0:Data_x=Data_x.drop([feat_list[i]],axis=1)
            knn=KNeighborsClassifier(n_neighbors=5)
            cv_scores=cross_val_score(knn,Data_x,Data_y,cv=5)
            F=self.f_list.count(1)
            #fit,acc,cnt,final_fitness,subset_list
            #适应度曲线，精确率，特征子集个数，最终适应度，特征子集
            fit.append(self.fitness)
            global acc,cnt
            acc = np.mean(cv_scores)
            cnt = F

            '''
            print ('ACC:',np.mean(cv_scores))
            print (F)
            print ('Fitness:',self.fitness)
            print (self.f_list)
            '''
    class pop():
        def __init__(self,res=None):
            self.res=res
            self.FoodSource=None
            self.Leader=None

        def ordering_pop(self):
            for r in self.res:
                r.Fitness()
            ress=sort_by_fitness(self.res)
            self.FoodSource=ress[0]
            self.Leader=ress[1]
            self.res=ress


    def init_pop(n):
        init_list=[]
        for i in range(n):
            feat_list=np.zeros(len(list(Data_x)))
            feat_list[0]=0.2
            mu=4
            for i in range(len(list(Data_x))-1):
                temp_feat=mu*feat_list[i]*(1-feat_list[i])
                if temp_feat>0.5:
                #if temp_feat>np.random.uniform(0,1):
                    feat_list[i+1]=1
                else:
                    feat_list[i+1]=0

            init_list.append(Salp(f_list=feat_list))

        return init_list

    def sort_by_fitness(lis):
        sorted_list=[]
        list1=[obj.fitness for obj in lis]
        while list1:
            minn=min(list1)
            for l in lis:
                if l.fitness==minn:
                    sorted_list.append(l)
                    lis.remove(l)
                    list1=[obj.fitness for obj in lis]
                    break
        return sorted_list

   
    def SALP(pop,epochs=50):
        a=2

        for ep in range(epochs):
            #print(f'{ep/epochs*100}%')
            pop.FoodSource.Acc()
            #print(pop.FoodSource.fitness)
            #print('')
            next_leader=Salp(f_list=np.zeros(len(list(Data_x))))
            next_res=[]
            s1 = a - ep*a / epochs
            r2=0.2
            for i in range(len(next_leader.f_list)):
                omega=0.7
                r1=2*np.exp(-1*(-6*ep/epochs)**2)
                #r2=np.random.uniform(0,1)
                r2=4*r2*(1-r2)
                r3=np.random.uniform(0,1)
                if r3>= 0.5:
                    temp_next=(omega*pop.FoodSource.f_list[i])+(r1*r2)
                    #next_leader.f_list[i]=temp_next+s1*math.cos(np.random.uniform(0,2*math.pi))*abs(np.random.uniform(0,1)*pop.FoodSource.f_list[i]-temp_next)
                if r3<  0.5:
                    temp_next=(omega*pop.FoodSource.f_list[i])-(r1*r2)
                    #next_leader.f_list[i]=temp_next+s1*math.sin(np.random.uniform(0,2*math.pi))*abs(np.random.uniform(0,1)*pop.FoodSource.f_list[i]-temp_next)
                #'''
                s4=np.random.uniform(0,1)
                if s4>= 0.5:next_leader.f_list[i]=temp_next+s1*math.cos(np.random.uniform(0,2*math.pi))*abs(np.random.uniform(0,1)*pop.FoodSource.f_list[i]-temp_next)
                if s4<  0.5:next_leader.f_list[i]=temp_next+s1*math.sin(np.random.uniform(0,2*math.pi))*abs(np.random.uniform(0,1)*pop.FoodSource.f_list[i]-temp_next)
                #'''
                next_leader.f_list[i]-=((omega)/2)
                next_leader.f_list[i]=1/(1+np.exp(-10*next_leader.f_list[i]))
                rand=np.random.uniform(0,1)
                if next_leader.f_list[i] >= rand : next_leader.f_list[i]=0
                else : next_leader.f_list[i]=1
            for m in range(len(pop.res[2:])):
                n_res=np.zeros(len(pop.res[m+2].f_list))
                for i in range(len(pop.res[m+2].f_list)):
                    n_res[i]=0.5*(pop.res[m+2].f_list[i]+(omega*pop.res[m+1].f_list[i]))
                    n_res[i]-=((1+omega)/4)
                    n_res[i]=1/(1+np.exp(-10*n_res[i]))
                    rand=np.random.uniform(0,1)
                    if n_res[i] >= rand : n_res[i]=0
                    else : n_res[i]=1   
                nn_res=[int(item) for item in n_res]
                next_res.append(Salp(f_list=nn_res))
            next_res.append(next_leader)
            next_res.append(pop.FoodSource)
            pop.res=next_res
            pop.ordering_pop()

    results=init_pop(20) 
    pop=pop(res=results)
    pop.ordering_pop()
    list1=[pop]
    a = datetime.datetime.now()  
    SALP(pop)
    b = datetime.datetime.now()
    c = b - a
    print( int(c.total_seconds()))
    list1=[pop]
    #np.savetxt('corn_rust_ssafs.csv',fit,delimiter=",")
    #适应度曲线 list
    print('fit_curve:',fit)
    #精确率 int
    print('acc:',acc)
    #特征子集个数 int
    print('No. of subset:',cnt)
    #最终适应度 float64
    print('final fitness:',pop.FoodSource.fitness)
    #特征子集 list
    print('subset:',pop.FoodSource.f_list)



428
fit_curve: [0.25419439866192967, 0.23830406609774266, 0.1599862957672539, 0.1568774859905505, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13670314617441306, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.13288793203433077, 0.12551622379714072, 0.12551622379714072, 0.12551622379714072, 0.12551622379714072, 0.12551622379714072, 0.12415077860796739, 0.12415077860796739, 0.12415077860796739, 0.12415077860796739, 0.12415077860796739,